![](https://s7280.pcdn.co/wp-content/uploads/2020/04/WhatIstheInternetofThings.png)

In [ ]:
import pandas as pd
import numpy as np

train = pd.read_csv("../input/iot-device-identification/iot_device_train.csv")
test = pd.read_csv("../input/iot-device-identification/iot_device_test.csv")

train.head()

In [ ]:
test.head()

## We can see both training data and testing data has same number of columns, we have to predict device category but the problem is column device category is in test data too. There will be 2 approaches

### 1) If the target column of both train and test have different number of unique values then we have to drop he extra categories or unique values to balance the data, but by that we may lose valuable information about a particular category if we drop it.
### 2) We can combine the two data and then we can solve the problem.

#### Let's see, shall we?

In [ ]:
train['device_category'].value_counts()

In [ ]:
test['device_category'].value_counts()

### We can clearly see water sensor is not in train data but present in test data

In [ ]:
print( train.shape)
print("-----------------")
print(test.shape)

1000 rows and 298 columns in train data

900 rows and 298 columns in test data

I would like the 2nd method more.

In [ ]:
frames = [train, test]
df = pd.concat(frames, ignore_index=True)
df= df.sample(frac = 1).reset_index(drop = True)
df.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
%matplotlib inline



df.hist(figsize=(40,90))
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder

from sklearn.ensemble import RandomForestClassifier

for c in df.columns:
    if df[c].dtype=='object': 
        lbl = LabelEncoder()
        lbl.fit(list(df[c].values))
        df[c] = lbl.transform(df[c].values)

In [ ]:
all_corr = df.corr().abs()['device_category'].sort_values(ascending = False)
all_corr

In [ ]:
corr_drop = all_corr[all_corr < 0.1]
corr_drop

In [ ]:
to_drop = list(corr_drop.index)

In [ ]:
df2 = df.drop(to_drop, axis = 1)

In [ ]:
df2

In [ ]:
X = df2.drop(['device_category'], axis = 1)
y  = df2['device_category']

In [ ]:
from sklearn.model_selection import train_test_split # For train/test splits

from sklearn.feature_selection import VarianceThreshold # Feature selector
from sklearn.pipeline import Pipeline # For setting up pipeline
# Various pre-processing steps
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV # For optimization
from sklearn.ensemble import GradientBoostingClassifier

X_train, X_test, y_train, y_test = train_test_split(X,  y, test_size=.3, random_state=0)
 
print(X_train.shape)
print(X_test.shape)

In [ ]:
#GridSearch for parameters
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning)
from xgboost import XGBClassifier

pipe = Pipeline([('selector', VarianceThreshold()),
                 ('classifier', XGBClassifier(eval_metric = 'auc', use_label_encoder=False,objective = 'multi:softprob'
                                             ))])

parameters = {
    'selector__threshold': [0,0.02,0.05,0.08,0.1],
     'classifier__eta' : [0.05,0.1, 0.3, 0.5,0.6,0.7,0.9],
	'classifier__min_child_weight': [1, 3,5, 8, 10],
	'classifier__max_depth': [2,4, 6, 8, 12],
    'classifier__n_estimators' : [50,100,200,400,40, 500, 600]
}

grid = RandomizedSearchCV(pipe, parameters, scoring = 'accuracy',verbose = 5).fit(X_train, y_train)

print('Test set score: ' + str(grid.score(X_test, y_test)))

## 89% accuracy

In [ ]:
# Access the best set of parameters
best_params = grid.best_params_
print(best_params)
# Stores the optimum model in best_pipe
best_pipe = grid.best_estimator_
print(best_pipe)

## Upvote if you like it or fork it, this helps us publish more notebooks for the community.